# AMPEL demo: Matching ZTF transients in Healpix map to model.

This notebook compares the core elements of comparing a model for the optical emission from a NS merger (a kilonova) with optical alerts form the ZTF survey found within the bounaries of a LIGO/Virgo Healpix map. Consists of three main segments:
- Defining a model for kilonova emission & matching this with a (random) ZTF transient.
- Finding a suitable Healpix map. 
- AMPEL processing: Setting up a local instance, retrieving optical data and matching these to catalogs and lightcurves.

Running this notebook requires a local AMPEL build. Quick conda-based install instructions:


In [ ]:
%%capture --no-display
import os, requests
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('seaborn-colorblind')
from IPython.display import SVG
from IPython.display import Image
import ampel_quick_import
%qi AmpelLogger T2RunPossis DevAmpelContext T0HealpixPathProcessor ZiCompilerOptions ZTFHealpixAlertLoader AmpelVault DictSecretProvider T2CatalogMatch 

In [ ]:
from util_tutorial import api_to_lightcurve
from util_tutorial import standard_configs

In [ ]:
# Access to the ZTF archive requires an access token:
# These are public and free to get, but required to monitor loads.  
token = os.environ["ARCHIVE_TOKEN"]
AMPEL_CONF = '/home/jnordin/github/ampelTutorial/ampel_conf_707140.yaml'

## I. A kilonova model

As an example of optical emission we will use a (random) one from the POSSIS library (Bulla et al, 2019). This can be used to predict the output in any optical band as a function of time since explosion.

In [ ]:
Image(url="https://oup.silverchair-cdn.com/oup/backfile/Content_public/Journal/mnras/489/4/10.1093_mnras_stz2495/2/m_stz2495fig5.jpeg?Expires=1653663985&Signature=iGZTSOeDkTa6uOZ1saQFtfyjkesfVWdVmg-fDx842Nui-~Qr~lYmNy7aL13RANasKWQ80EFV3IUuK~OoysAAvjJxmmvyY0wwL6pFr5~fRSteXi7~3xPv0LypDf3VOIDzCnXYcbjsQndp8i2dWYi6oCAIG6MKrgLmVLNIbW3FOFL8eBxGTjVZLBpoWjwDccaiNCekORboDJMO62guW863UNqbyPilt~Dvagmg3B-UcadL16C0fin-rWL78Z0P73t0Y71s~WZs05xkf1c63Vpbysj1zuXYnuW3f7zckTsuLMXZluBq40GUqg65qYzlCO25qRAQ8sNEDxLpARzq1-Kk2Q__&Key-Pair-Id=APKAIE5G5CRDK6RD3PGA")

In [ ]:
# Initialize a wrapper method which retrieves the model (delay)
t2 = T2RunPossis(backup_z=0.03, logger=AmpelLogger.get_logger(), t2_dependency=[], plot_props=None)
t2.post_init()

In [ ]:
# Plot the model lightcurve (for arbitrary parameters)
t = np.arange(0,4,0.4)
for b in ['ztfg','ztfr']:
    plt.plot(t,t2.sncosmo_model.bandflux(b, t),label=b,linewidth=4)
plt.xlabel('Time since explosion (days)')
plt.ylabel('Flux')
plt.legend()
plt.show()

In [ ]:
# Retrieve optical data for a (named) transient and fit to kilonova model.
snname = "ZTF19aakssbm"

In [ ]:
lc = api_to_lightcurve(snname, token=token)

In [ ]:
t2out = t2.process(lc, t2_views=[])

In [ ]:
t2.sncosmo_model.parameters = t2out['sncosmo_result']['parameters']

In [ ]:
t = np.array([0,0.2,0.5,1,1.5, 2,3]) + t2.sncosmo_model.parameters[1]
for b in ['ztfg','ztfr']:
    plt.plot(t,t2.sncosmo_model.bandmag(b, 'ab', t),label=b,linewidth=5)
for fname, fid in zip(["Obs ZTF g", "Obs ZTF R"],[1,2]):
    phot = np.array(lc.get_ntuples(['jd', 'magpsf', 'sigmapsf'], filters={'attribute': 'fid', 'operator': '==', 'value': fid}))
    plt.errorbar(phot[:,0],phot[:,1],yerr=phot[:,2],fmt='.', markersize=20, label=fname)
plt.xlabel('Time (JD)')
plt.ylabel('Magnitude')
plt.xlim([t[0]-3,t[-1]+6])
plt.gca().invert_yaxis()
plt.legend()
plt.show()

## II. A GW contour

We will here use the published map of S200213t as a sample detection region with reasonable size. 

In [ ]:
Image(url="https://gracedb.ligo.org/api/superevents/S200213t/files/LALInference.png")

In [ ]:
# Define name and path to healpix map, as well as a P-value threshold:
map_name = 'S200213t'  # Some cand, ok  no reasonable cuts
map_url = f"https://gracedb.ligo.org/api/superevents/{map_name}/files/LALInference.fits.gz"
pvalue_limit = 0.9

# So what do I do now?

I have a merger model and I have a GW signal - what is the next step?
We can sketch a scientific analysis:
- Get hold of a lot of data coming from this (fairly large) region in an efficient fashion.
- Go through a galaxy catalogs to find distances, where available.
- Compare all of these matches with the merger model, and see whether any are compatible with explosion times and distances in a statistical sense.
- Set this up to match all X models, all X template matches or add it to a live stream of alerts or...

This seems like a lot of work, and work which someone else is probably better at...


## The AMPEL framework

We can now identify units which can complete these steps:

In [ ]:
# Already used one interface unit which loaded a particular POSSIS model and made it accessible through
# a generalized lightcurve format.
T2RunPossis??

In [ ]:
# There is also a general method for matching coordinates to any in a large series of catalgs
T2CatalogMatch??

In [ ]:
# And a processor which systematically chunks a healpix maps and retrieves alerts for you
T0HealpixPathProcessor??

In [ ]:
# The output data will be stored in an (assumed existing) local MongoDB. 
# Also, a scratchdir is used to store plots
dbname = 'HereusDB'
channel = 'Tutorial'
scratchdir = '/home/jnordin/tmp/tutorial'

In [ ]:
%%capture --no-display
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = dbname,
    purge_db = True,
    vault = AmpelVault([DictSecretProvider({"ztf/archive/token": os.environ["ARCHIVE_TOKEN"],})]),
)
ctx.add_channel(
    name=channel,
    access=['ZTF', 'ZTF_PUB', 'ZTF_PRIV']
)

The most complex part of an alert schema are the directives which regulate which units should be applied to which datapoints of which streams:

In [ ]:
directives = {'channel': 'Tutorial',
    'filter': {
                'unit': 'DecentFilter',
                'config': standard_configs('DecentFilter', 'Single'),
                'on_stock_match': 'bypass'},
    'ingest': {
                'stock_t2': [{'unit': 'T2PropagateStockInfo',
                'config': standard_configs('T2PropagateStockInfo', 'Healpix')}],
                'mux': {'unit': 'ZiArchiveMuxer',
                        'config': {'history_days': 50, 'future_days': 50},
                        'combine': [{'unit': 'ZiT1Combiner',
                        'state_t2': [ {
                                        'unit': 'T2DigestRedshifts',
                                        'config': standard_configs('T2DigestRedshifts', 'MultipleFirstPPSPhotoZ')},
                                    {'unit': 'T2RunPossis', 'config': standard_configs('T2RunPossis', 'StockTriggerPhotoZ')}]}
                                   ],
                        'insert': {'point_t2': 
                                   [{'unit': 'T2CatalogMatch', 'config': standard_configs('T2CatalogMatch', 'MultipleFirstPPS'),
                                    'ingest': {'filter': 'PPSFilter',
                                    'sort': 'jd',
                                    'select': 'first'}}]
                                  }
                       }
    }
}
directives['ingest']['mux']['combine'][0]['state_t2'][1]['config']['plot_props']['disk_save'] = scratchdir

All of the above os joined in a processor which combined the data streams and allocates tickets for future operations.

In [ ]:
ac = T0HealpixPathProcessor(
    context = ctx,
    process_name = "LIGOScript_{}".format(channel),
    iter_max = 100000,
    map_name = map_name,
    map_url = map_url,
    scratch_dir = scratchdir,
    pvalue_limit = pvalue_limit,
    supplier = {
        "unit": "ZiHealpixAlertSupplier",
        'config': {
            'loader': {
                'unit': 'ZTFHealpixAlertLoader',
                'config':{
                    'archive_token': os.environ["ARCHIVE_TOKEN"],
                    'chunk_size': 1000,
                    'future_days': 3,
                    'history_days': 10,
                }
            },
        }
    },
    shaper = "ZiDataPointShaper",
    log_profile = "debug",
    compiler_opts = ZiCompilerOptions(),
    database = "mongo",
    directives = directives,
)
ac.run()

In [ ]:
%%capture --no-display
t2w = ctx.new_context_unit(
    unit = 'T2Worker',
    process_name = 'T2Processor_7',
)
t2w.run()

## Summary of what just happened

This notebook skips layers of details regarding what happens. This is a summary:

- Read an arbitrary kilonova model into an interface unit.
- Download a Healpix map from GraceDB.
- Retrieved optical data from potential ZTF counterparts within the 90% contour in bunches.
- Filtered for likely astrophysical events, avoiding stars from Gaia/PanStarrs and artifacts.
- Matched to a series of catalogs, looking for redshifts (spectroscopic of available, otherwise photo-z).
- Fit a kilonva model fixed to the GraceDB explosion time and whatever redshift information was available.

A complex process, but it is something which you can run (and improve on).

In [ ]:
# Lets look at some (almost random) outcome
SVG(filename="{}/ZTF20aanlgon.svg".format(scratchdir))

Did we just find a kilonova? Not likely. But we did construct an analysis schema which can be improved, distributed, applied to a large samples or turned into a real-time analysis. 

### Takeaway message

- Low-latency multi-messenger astronomy will be exciting but also hard: tools, data-streams and models need to be modluar and learn to talk to each other in an efficient and repeatable way. 
- At the other hand, the same system will open up a wealth of possibilities for creative scientists who comes up with a way to improve one component...
- ... if we make sure that what we produce can be accessed and used by others. 
- AMPEL is a public framework which can allow you to share models, parse data or construct real-time programs.